# 1) librairies

In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import nltk
nltk.download('wordnet')
nltk.download('punkt')
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import  TweetTokenizer, sent_tokenize
# Libraries for text preprocessing
import re
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

import nltk.data
from gensim.models import word2vec
from gensim.models import KeyedVectors
from collections import OrderedDict
#!pip install wordcloud

#Word cloud
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer


from scipy import sparse
#Function for sorting tf_idf in descending order
from scipy.sparse import coo_matrix

!pip install rake_nltk
!pip install yake
from rake_nltk import Rake
import yake
from gensim.summarization import keywords
import spacy
import operator
from rake_nltk import Rake
from sklearn.decomposition import PCA



[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
  Created wheel for rake-nltk: filename=rake_nltk-1.0.4-py2.py3-none-any.whl size=7819 sha256=b26fbc6839798a33bae44ad7c0a48b1fc9a83ef2044d40b075c061bc875f3717
  Stored in directory: /root/.cache/pip/wheels/ef/92/fc/271b3709e71a96ffe934b27818946b795ac6b9b8ff8682483f
Successfully built rake-nltk
     |████████████████████████████████| 61kB 3.6MB/s 
     |████████████████████████████████| 92kB 4.5MB/s 
  Created wheel for segtok: filename=segtok-1.5.10-cp37-none-any.whl size=25019 sha256=cc0a6656dd2ee63e8adcdf8ed46360cecedc8196b64db3e080aaddc9a6a8e96a
  Stored in directory: /root/.cache/pip/wheels/b4/39/f6/9ca1c5cabde964d728023b5751c3a206a5c8cc40252321fb6b
Successfully b

# 2) Preprocessing

In [35]:
#df = pd.read_csv("/Users/eliaschaumeix/Desktop/3A/S2/NLP/df.csv")
df = pd.read_csv("df.csv")

acta_literaria = df.loc[:,['abstract_en','keywords_en']].dropna()
acta_literaria.info()
acta_literaria = acta_literaria.reset_index()

n = acta_literaria.shape[0]


lem = WordNetLemmatizer()
stem = PorterStemmer()

##Creating a list of stop words and adding custom stopwords
stop_words = set(stopwords.words("english"))
##Creating a list of custom stopwords
new_words = ["using", "show", "result", "large", "also", "iv", "one", "two", "new", "previously", "shown", "abstract"]
stop_words = stop_words.union(new_words)

acta_literaria['abstract_sentences'] = ""
tokenizer_words = TweetTokenizer()
for i in range (len(acta_literaria)):
    tokens_sentences = [tokenizer_words.tokenize(t) for t in nltk.sent_tokenize(acta_literaria['abstract_en'][i])]
    acta_literaria['abstract_sentences'][i] = tokens_sentences

x = acta_literaria['abstract_sentences'].copy()
for i in range(acta_literaria.shape[0]) : 
    sentences = x[i]
    new = []
    for sentence in sentences :
        sentence_str = ' '.join(sentence)
        new.append(sentence_str)
    x[i] = new
acta_literaria['sentence'] = ""
acta_literaria['sentence'] = x

corpus = []
for i in range(n):
    #Remove punctuations
    text = re.sub('[^a-zA-Z]', ' ', acta_literaria['abstract_en'][i])
    
    #Convert to lowercase
    text = text.lower()
    
    #remove tags
    text=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",text)
    
    # remove special characters and digits
    text=re.sub("(\\d|\\W)+"," ",text)
    
    ##Convert to list from string
    text = text.split()
    
    ##Stemming
    ps=PorterStemmer()
    #Lemmatisation
    lem = WordNetLemmatizer()
    text = [lem.lemmatize(word) for word in text if not word in  
            stop_words]
    text = [lem.lemmatize(word,'v') for word in text if not word in  
            stop_words]
    text = " ".join(text)
    corpus.append(text)

corpus1 = []
for i in range(0, acta_literaria.shape[0]):
    res  = []
    for element in acta_literaria['sentence'][i] :
        #Remove punctuations
        text = re.sub('[^a-zA-Z]', ' ', element)
        
        #Convert to lowercase
        text = text.lower()
        
        #remove tags
        text=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",text)
        
        # remove special characters and digits
        text=re.sub("(\\d|\\W)+"," ",text)
        
        ##Convert to list from string
        text = text.split()
        
        ##Stemming
        ps=PorterStemmer()
        #Lemmatisation
        lem = WordNetLemmatizer()
        text = [lem.lemmatize(word) for word in text if not word in  
                stop_words]
        text = [lem.lemmatize(word,'v') for word in text if not word in  
                stop_words]
        text = " ".join(text)
        res.append(text)
    corpus1.append(res)

corpus2 = []
for abstract in corpus1 :
    res = []
    for sentence in abstract :
        res.append(sentence.split(' '))
    corpus2.append(res)
corpus3 = []
for i in range(0, acta_literaria.shape[0]):
    res  = []
    for element in acta_literaria['sentence'][i] :
        #Remove punctuations
        text = re.sub('[^a-zA-Z]', ' ', element)
        
        #Convert to lowercase
        text = text.lower()
        
        #remove tags
        text=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",text)
        
        # remove special characters and digits
        text=re.sub("(\\d|\\W)+"," ",text)
        
        ##Convert to list from string
        text = text.split()
        
        text = " ".join(text)
        res.append(text)
    corpus3.append(res)
corpus4 = []

for abstract in corpus3 :
    res = []
    for sentence in abstract :
        res.append(sentence.split(' '))
    corpus4.append(res)

X = pd.DataFrame({"corpus" : corpus, "corpus1" : corpus1, "corpus2" : corpus2, "corpus3" : corpus3, "corpus4" : corpus4 })

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37 entries, 1 to 40
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   abstract_en  37 non-null     object
 1   keywords_en  37 non-null     object
dtypes: object(2)
memory usage: 888.0+ bytes


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


# 3) Construction of the matrix of scores

diverses foncitons à commenter

In [36]:
cv=CountVectorizer(max_df=0.8,stop_words=stop_words, max_features=10000, ngram_range=(1,1))
cv_corpus=cv.fit_transform(X.corpus)


tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(cv_corpus)
# get feature names
feature_names=cv.get_feature_names()


def extract_tf_idf_score(feature_names, items):
      """get the feature names and tf-idf score"""

  
      score_vals = []
      feature_vals = []
      
      # word index and corresponding tf-idf score
      for idx, score in items:
          
          #keep track of feature name and its corresponding score
          score_vals.append(round(score, 3))
          feature_vals.append(feature_names[idx])
  
      #create a tuples of feature,score
      #results = zip(feature_vals,score_vals)
      results= {}
      for idx in range(len(feature_vals)):
          results[feature_vals[idx]]=score_vals[idx]
      
      return results

Text rank 

In [37]:
class TextRank4Keyword():
    """Extract keywords from text"""
    
    def __init__(self, abstract, d =  0.85, min_diff = 1e-5, steps = 10, node_weight = None, crit = 0.8):
        self.d = d # damping coefficient, usually is .85
        self.min_diff = min_diff # convergence threshold
        self.steps = steps # iteration steps
        self.node_weight = node_weight # save keywords and its weight
        self.crit = crit
        self.abstract = abstract
        
    def get_vocab(self):
        """Get all tokens"""
        vocab = OrderedDict()
        vocab1 = {}
        i = 0
        for sentence in self.abstract:
            for word in self.abstract:
                if word not in vocab:
                    vocab[word] = i
                    i += 1
                    vocab1[word] = 1
                else : 
                    vocab1[word] += 1
                    
        return vocab, vocab1
    
    def get_token_pairs(self, window_size):
        """Build token_pairs from windows in sentences"""
        token_pairs = list()
        for sentence in self.abstract:
            for i, word in enumerate(sentence):
                for j in range(i+1, i+window_size):
                    if j >= len(sentence):
                        break
                    pair = (word, sentence[j])
                    if pair not in token_pairs:
                        token_pairs.append(pair)
        return token_pairs
        
    def symmetrize(self, a):
        return a + a.T - np.diag(a.diagonal())
    
    def get_matrix(self, vocab, token_pairs):
        """Get normalized matrix"""
        # Build matrix
        vocab_size = len(vocab)
        g = np.zeros((vocab_size, vocab_size), dtype='float')
        for word1, word2 in token_pairs:
            i, j = vocab[word1], vocab[word2]
            g[i][j] = 1
            
        # Get Symmeric matrix
        g = self.symmetrize(g)
        
        # Normalize matrix by column
        norm = np.sum(g, axis=0)
        g_norm = np.divide(g, norm, where=norm!=0) # this is ignore the 0 element in norm
        
        return g_norm

    
    def get_keywords(self, number=10):
        """Print top number keywords"""
        node_weight = OrderedDict(sorted(self.node_weight.items(), key=lambda t: t[1], reverse=True))
        for i, (key, value) in enumerate(node_weight.items()):
            print(key + ' - ' + str(value))
            if i > number:
                break
        
    def get_scores(self, number=10):
        """Print top number keywords"""
        node_weight = OrderedDict(sorted(self.node_weight.items(), key=lambda t: t[1], reverse=True))
        keywords = {}
        for i, (key, value) in enumerate(node_weight.items()):
            print(key + ' - ' + str(value))
            keywords[key] = value
        return(keywords)
            

    def analyze(self,
                window_size=4):
        """Main function to analyze text"""
        
        
        
       
        
        # Build vocabulary
        vocab, vocab1 = self.get_vocab(self.abstract)
        
        self.n_tot = 0
        for key in vocab1.keys():
            self.n_tot += vocab1[key]
                
        #Get rid of too occurent words
        self.remove_list = []
        for key in vocab1.keys():
            if vocab1[key]/self.n_tot > self.crit :
                vocab.pop(key)
                self.remove_list.append(key)
        
        # Get token_pairs from windows
        token_pairs = self.get_token_pairs(window_size, corpus2[0])
        
        # Get normalized matrix
        g = self.get_matrix(vocab, token_pairs)
        
        # Initionlization for weight(pagerank value)
        pr = np.array([1] * len(vocab))
        
        # Iteration
        previous_pr = 0
        for epoch in range(self.steps):
            pr = (1-self.d) + self.d * np.dot(g, pr)
            if abs(previous_pr - sum(pr))  < self.min_diff:
                break
            else:
                previous_pr = sum(pr)

        # Get weight for each node
        node_weight = dict()
        for word, index in vocab.items():
            node_weight[word] = pr[index]
        
        self.node_weight = node_weight

fonction pour textrank w2v

In [38]:
def cosine_text_rank(w2v_mat) : 
  # similarity matrix, it is 30 times faster for sparse matrix
  # replace this with A.dot(A.T).todense() for sparse representation
  similarity = np.dot(w2v_mat, w2v_mat.T)

  # squared magnitude of preference vectors (number of occurrences)
  square_mag = np.diag(similarity)

  # inverse squared magnitude
  inv_square_mag = 1 / square_mag

  # if it doesn't occur, set it's inverse magnitude to zero (instead of inf)
  inv_square_mag[np.isinf(inv_square_mag)] = 0

  # inverse of the magnitude
  inv_mag = np.sqrt(inv_square_mag)

  # cosine similarity (elementwise multiply by inverse magnitudes)
  cosine = similarity * inv_mag
  cosine = cosine.T * inv_mag
  return(cosine)

# pagerank powermethod
def powerMethod(A, m, iter):
    n = A.shape[1]
    delta = m * (np.array([1] * n, dtype='float64') / n)
    for i in range(iter):
        x0 = np.dot((1 - m), np.dot(A, x0)) + delta
    return x0

Word 2 vec pretrained sur wikipedia

In [60]:
from google.colab import files
uploaded = files.upload()

KeyboardInterrupt: ignored

In [52]:
w2v = KeyedVectors.load_word2vec_format('C:/Users/Tom/Desktop/Cours/Ensae cours/nlp/projet/enwiki_20180420_100d.txt', binary=False)

FileNotFoundError: ignored

In [ ]:
n_w2v_proj = 10
n_w2v_tot = 100
m = 0.15
iter = 130
data_final = pd.DataFrame()
for i in range (n) : 

  #TF IDF SCORE

  tf_idf_vector=tfidf_transformer.transform(cv.transform([X.corpus[i]]))

  items=tuples = zip(tf_idf_vector.tocoo().col, tf_idf_vector.tocoo().data)
  tf_idf = extract_tf_idf_score(feature_names,items)
  
  # tf_idf est un dico qui contient mots : tf_idf_score

  #TextRank
  t4r = TextRank4Keyword(corpus2[i])
  t4r.analyze()
  text_rank = t4r.get_scores()

  # Word2vec entraîné sur wikipédia 
  w2v_list = []

  for word in tf_idf.keys() :
    w2v_list.append(w2v(word))

  w2v_mat = np.array(w2v_list).reshape(len(tf_idf), n_w2v_tot)
  pca = PCA(n_components = n_w2v_proj)

  w2v_proj = pca.fit_transform(w2v_mat)
  cosine = cosine_text_rank(w2v_mat)

  x_0 = [1] * cosine.shape[1] 

  w2v_weights = powerMethod(cosine, x_0, m, iter)
  w2v_weights = ( w2v_weights - w2v_weights.min() ) / ( w2v_weights.max() - w2v_weights.min() )

  # RAKE 
  rake_fct = Rake(stopwords = stop_words) 
  rake_fct.extract_keywords_from_text(acta_literaria['abstract_en'][i])
  degree = rake_fct.get_word_degrees()
  freq = rake_fct.get_word_frequecy_distribution()
  rake_score = {}

  for word in tf_idf.keys() :
    rake_score[word] = degree[word] / freq[word]

  # YAKE! 

  max_ngram = 1 
  d = 0.85
  n_key = len(tf_idf)
  lang = 'en'
  yake_fct = yake.KeywordExtractor(lan = lang, n = max_ngram, dedupLim = d, top = n_key, features = None, stopwords = stop_word)
  yake_kw_list = yake_fct.extract_keywords(acta_literaria['abstract_en'][i])
  yake_kw = {yake_kw_list[i][0] : yake_kw_list[i][1] for i in range(len(yake_kw_list))}
  data_score = {}
  j = 0
  for word in tf_idf.keys() : 
    data_score[word] = [tf_idf[word], yake_kw[word], rake_score[word], text_rank[word], w2v_weights[j] ] + w2v_proj[j, :].tolist() + [i]
    j+=1

  df_step = pd.DataFrame(data_score).T
  data_final = pd.concat([data_final, df_step])


In [18]:
import numpy as np
np.array([[1,2],[1,2]]).tolist()

[[1, 2], [1, 2]]

In [25]:
import pandas as pd
a = pd.DataFrame({'a':[1,2],'b':[1,2]}).T
b = pd.DataFrame({'a':[3,4],'b':[3,4]}).T

In [28]:
pd.concat([a,b])

,0,1
a,1,2
b,1,2
a,3,4
b,3,4


In [30]:
pd.concat([pd.DataFrame(),a])

,0,1
a,1,2
b,1,2
